In [52]:
import pandas as pd
import datetime
from tqdm import tnrange

In [2]:
userLog = pd.read_csv("./clean_data/userlog_feature.csv")

In [3]:
userLog['datetime'] = pd.to_datetime(userLog['datetime'])
isTrain = userLog['datetime']<datetime.datetime(2018,2,25)
test = userLog[~isTrain]
userLog = userLog[isTrain]


In [4]:
#testUserAnswerMerged.to_csv("./clean_data/_test_with_train.csv")

In [5]:
userLog1 = userLog.sort_values("datetime")
userLog1.sample()

,userCode,project_id,requestedDevice,userAgent,pageReferrer,year,month,day,hour,datetime,user_clicked_by_hour,user_clicked_projects_by_hour
1080082,de89bac5-57c6-ecfb-184d-cc4e973c31ac,6021,Desktop,Other_OS,Other_PageReferer,2018,2,17,7,2018-02-17 07:00:00,23,2


In [6]:
userLog2VecDf = userLog1.groupby("userCode").agg({"project_id":lambda x: list(x)})

In [7]:
#userLog2VecDf.sample(10)
userLog2VecDf.shape

(482751, 1)

In [8]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [9]:
#userLog2VecDf['project_id'].apply(len).value_counts()#.plot(kind="kde")

In [10]:
moreThanOne = userLog2VecDf['project_id'].apply(len)>1
projs = userLog2VecDf['project_id'][moreThanOne]
projs = [[str(p) for p in proj] for proj in projs]
users = userLog2VecDf.index[moreThanOne]


In [11]:
documents = [TaggedDocument(proj, [user]) for i, (proj,user) in enumerate(zip(projs,users))]

In [12]:
documents[:10]

[TaggedDocument(words=['6709', '6712'], tags=['0000e1e2-f595-0ae7-860f-fcc07dcb116e']),
 TaggedDocument(words=['8829', '4703', '6577', '4703', '8829', '4184'], tags=['0000fa46-1f0b-9504-b568-43479d17620e']),
 TaggedDocument(words=['8587', '5985', '9095'], tags=['00019a11-0aaa-2559-4feb-534724830d3e']),
 TaggedDocument(words=['6070', '6070', '6070', '6070'], tags=['0002abb4-b370-d047-0d70-bdfa59780107']),
 TaggedDocument(words=['8412', '8412', '5452', '5858', '7591', '4411', '3738', '4980', '7128'], tags=['0002d107-28fe-ffb0-2ebf-a7fb1fdb8a3f']),
 TaggedDocument(words=['1828', '1828'], tags=['00031c31-5c7b-563b-4298-f1b320bfea6f']),
 TaggedDocument(words=['3839', '3839'], tags=['00033b74-b212-b7b1-bef1-38b21d2097b7']),
 TaggedDocument(words=['9333', '9333'], tags=['00039df0-f294-5bd1-93d1-7ec49b27940b']),
 TaggedDocument(words=['8631', '8742', '9334', '8742', '9334'], tags=['0003d990-37d2-140c-a68e-060bc1e81ea5']),
 TaggedDocument(words=['6545', '3522', '6545'], tags=['00040647-ef7a-936

### Train

In [13]:
model = Doc2Vec(documents, vector_size=300, window=3, min_count=1, workers=4,epochs=4,sample=1e-4, negative=10)

### Let's see if it works or not

In [24]:
# def recommendByVector(userId):
#     try:
#     #print(most_similar_user)
#         seen = userLog2VecDf.loc[userId].project_id
#         most_similar_user = model.docvecs.most_similar(userId,topn=100)
#         recommending_projs = []
#         for user, similarity in most_similar_user:
#             friend_projects = [i for i in userLog2VecDf.loc[user].project_id if i not in seen]
#             recommending_projs.extend(friend_projects)
#             recommending_projs = list(set(recommending_projs))
#             if len(recommending_projs)>=7: break

#         #     print(userLog2VecDf.loc[user].project_id, similarity)

#         return " ".join([str(i) for i in recommending_projs])
#     except:
#         return "5858 3738 4201 5452 5678 5764 5067"
    
# recommendByVector("0018fd5f-213f-c9ab-ebf4-0dcfb63be43b")
userLog.head()

,userCode,project_id,requestedDevice,userAgent,pageReferrer,year,month,day,hour,datetime,user_clicked_by_hour,user_clicked_projects_by_hour
0,7717bdc2-ea3e-e8ad-5d6b-178bd71c38b2,7956,Mobile,Android,HomeWebsite,2018,1,1,0,2018-01-01,3,3
1,7717bdc2-ea3e-e8ad-5d6b-178bd71c38b2,7956,Mobile,Android,HomeWebsite,2018,1,1,0,2018-01-01,3,3
2,cb5b4b68-cc01-6db6-f54b-4a0f881301c5,5067,Mobile,iPhone,HomeWebsite,2018,1,1,0,2018-01-01,1,1
3,5f74cef2-0d1e-b619-3564-0955a14e0985,6654,Mobile,iPhone,Google,2018,1,1,0,2018-01-01,1,1
4,dba8f279-844e-eef6-73ac-22bd7d1353cc,6474,Mobile,iPad,Google,2018,1,1,0,2018-01-01,2,2


In [32]:
userId = "7717bdc2-ea3e-e8ad-5d6b-178bd71c38b2"
userLog[userLog['userCode']==userId].project_id

0    7956
1    7956
7    7956
Name: project_id, dtype: int64

In [40]:
all_project = model.wv.vocab
def recommendByProjects(userId, userLog=userLog,topn = 50, withProb = False):
    seen = userLog[userLog['userCode']==userId].project_id
    
    seen = [str(i) for i in seen if str(i) in all_project]
    if len(seen) ==0: return "5858 3738 4201 5452 5678 5764 5067"
    most_similar_proj = model.wv.most_similar(positive=seen,topn=100)
    recommending_projs = []
    if withProb: return most_similar_proj
    
    for proj, similarity in most_similar_proj:
        if proj not in seen: recommending_projs.append(proj)
        if len(recommending_projs)>=topn: break

    #     print(userLog.loc[user].project_id, similarity)
    
    
    return " ".join([str(i) for i in recommending_projs])

recommendByProjects("7717bdc2-ea3e-e8ad-5d6b-178bd71c38b2", withProb =False)

/home/users/u5722792744_1/miniconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


'3054 3546 8281 5721 4728 4140 4812 3418 6091 3706 6544 4151 5820 5603 3805 5291 5150 3462 6517 5058 5032 9264 8465 3549 3943 4776 4007 5827 6581 3256 1782 3675 8726 5391 6575 4859 3252 8078 5596 6504 8525 4703 3821 6577 5708 6564 6539 5563 4025 6045'

## Test and evaluation

In [41]:
testUserAnswer = test.groupby("userCode").agg({"project_id":lambda x: list(x)})
testUserAnswer.columns = ["ans_project_list"]
testUserAnswerMerged = userLog2VecDf.merge(testUserAnswer,on='userCode',right_index=True)

In [53]:
scores = []
for i in tnrange(5):
    testing_users = pd.read_csv("./clean_data/test_data/test_before_021718_7000row_fold{}.csv".format(i))
    testing_users.head()

    testing_users['project_id'] = testing_users['userCode'].apply(recommendByProjects)
    testing_users['ans_project_id'] = testing_users['ans_project_id'].apply(eval)
    #testing_users['ans_project'] = testing_users['ans_project_id'].apply(lambda x: list(set(x)))
    testing_users['MAP'] = testing_users.apply(lambda x: score_AP(x['ans_project_id'],x['project_id']),axis=1)
    score = testing_users['MAP'].mean()
    print(score)
    scores.append(score)
print(sum(scores)/len(scores))  #0.04469187

HBox(children=(IntProgress(value=0, max=5), HTML(value='')))

/home/users/u5722792744_1/miniconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


KeyboardInterrupt: 

In [ ]:
testing_users.head()

In [ ]:
print("Accuracy: ",testing['MAP'].apply(lambda x: x>0).sum()/testing.shape[0])

In [ ]:
def score_AP(correct, recommended, n=7):
    recommended = recommended.split(" ")
    assert(len(set(recommended))==len(recommended)) # Recommended should be unique    
    precisions = []
    correct_score = 1
    for idx, recommend_item in enumerate(recommended):
        try:
            if int(recommend_item) not in correct: precisions.append(0)
            else:                         
                precisions.append(correct_score/(idx+1))
                correct_score+=1
        except:
            print(recommend_item)
    return sum(precisions)/min(len(correct),n)
#score_AP(["5858", "5678"],"5678 5858 8058 8107 6527 9132 6446",7)

In [ ]:
testing.head().apply(lambda x: score_AP(x['ans_project_list'],x['preds']),axis=1)

In [ ]:
submission_user = pd.read_csv("./data/testing_users.csv")
userLogForSubmit = pd.read_csv("./clean_data/userlog_feature.csv")
userLogNewest = userLogForSubmit.groupby("userCode").agg({"project_id":lambda x: list(x)})

In [ ]:
submission_user['project_id'] = submission_user['userCode'].apply(lambda x: recommendByProjects(x,userLogNewest))

In [ ]:
submission_user.head()

In [ ]:
submission_user.to_csv("./submission/item2vec_v4.csv",index=False)

In [ ]:
model.save("./model/doc2vec_before_train.wv")